# Core Imports

In [1]:
# Custom Imports
from polysaccharide import general
from polysaccharide.general import optional_in_place
from polysaccharide.extratypes import ResidueSmarts

from polysaccharide.molutils import reactions
from polysaccharide.molutils.rdmol.rdtypes import *
from polysaccharide.molutils.rdmol import rdcompare, rdconvert, rdkdraw, rdcompare, rdprops, rdbond, rdlabels

from polysaccharide.polymer import monomer as monoutils
from polysaccharide.polymer.monomer import MonomerInfo
from polysaccharide.polymer.management import PolymerManager

from polysaccharide.polymer import building
import mbuild as mb

# Generic Imports
import re
from functools import partial, cached_property
from collections import defaultdict
from itertools import combinations, chain
from ast import literal_eval

# Numeric imports
import pandas as pd
import numpy as np

# File I/O
from pathlib import Path
import csv, json, openpyxl

# Typing and Subclassing
from typing import Any, Callable, ClassVar, Generator, Iterable, Optional, Union
from dataclasses import dataclass, field
from abc import ABC, abstractmethod, abstractproperty
from openmm.unit import Unit, Quantity

# Cheminformatics
from rdkit import Chem
from rdkit.Chem import rdChemReactions

from openff.toolkit import ForceField
from openff.toolkit.topology import Topology, Molecule

# Static Paths
RAW_DATA_PATH  = Path('raw_monomer_data')
PROC_DATA_PATH = Path('processed_monomer_data')
RXN_FILES_PATH = Path('rxn_smarts')
MONO_INFO_DIR  = Path('monomer_files')

/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Wa

# File and chemistry type definitions

In [2]:
pdb_path = Path('pdb_files')
pdb_path.mkdir(exist_ok=True)

coll_path = Path('Collections')
coll_path.mkdir(exist_ok=True)

lammps_path = Path('LAMMPS')
lammps_path.mkdir(exist_ok=True)

In [3]:
# defining reacting functional groups
reaction_pairs = {
    'NIPU' : ('cyclocarbonate', 'amine'),
    'urethane' : ('isocyanate', 'hydroxyl')
}
# chemistries = ('urethane', 'NIPU')
chemistries = [i for i in reaction_pairs.keys()]

# Building PDB files from monomer fragments

In [ ]:
from polysaccharide.polymer.exceptions import SubstructMatchFailedError

DOP = 10

failed = general.RecursiveDict()
for chemistry in chemistries:
    mono_src_dir = MONO_INFO_DIR / chemistry
    pdb_out_dir = pdb_path / chemistry
    pdb_out_dir.mkdir(exist_ok=True)

    for mono_path in mono_src_dir.iterdir():
        mono_info = MonomerInfo.from_file(mono_path)
        try:
            polymer = building.build_linear_polymer(mono_info.SMARTS, DOP=DOP, sequence='AB')  
            savepath = pdb_out_dir / f'{mono_path.stem}.pdb'
            polymer.save(general.asstrpath(savepath))
        except Exception as e:
            failed[chemistry][e.__class__.__name__][mono_path.stem] = mono_info

In [ ]:
failed

## Testing sanitization of monomers prior to mbuild conversion

In [ ]:
rdkdraw.set_rdkdraw_size(500, 3/2)
cvtr = rdconvert.SMILESConverter()

chemistry = 'urethane'
idx = 10

mono_info = MonomerInfo.from_file(MONO_INFO_DIR / chemistry / f'{chemistry}_{idx}.json')

mbmols = {}
def show(mono_info : MonomerInfo) -> None:
    for resname, smarts in mono_info.SMARTS.items():
        print(resname, smarts)
        mol = Chem.MolFromSmarts(smarts)
        mol = cvtr.convert(mol)
        Chem.SanitizeMol(mol)
        rdlabels.clear_atom_map_nums(mol, in_place=True)
        display(mol)

        # mbmol, ports = building.mbmol_from_mono_smarts(new_smarts)

        prot_mol = rdbond.hydrogenate_rdmol_ports(mol, in_place=False)
        mbmols[resname] = mb.conversion.from_rdkit(prot_mol)

show(mono_info)

# Collating urethanes into collections and generating Interchange files

## Populating collections from newly-generated PDBs and monomers

In [ ]:
for chemistry in chemistries:
    chem_path = coll_path / chemistry
    chem_path.mkdir(exist_ok=True)

    mgr = PolymerManager(chem_path)
    mgr.populate_collection(pdb_path / chemistry, MONO_INFO_DIR / chemistry)

## Generating LAMMPS files via Interchange for both collections

In [4]:
from polysaccharide import OPENFF_DIR
from polysaccharide.charging.application import MolCharger

# specify forcefield
ff_name = 'openff-2.0.0.offxml'
ff_path = OPENFF_DIR / ff_name
forcefield = ForceField(ff_path)

# specify charging method
chg_method = 'Espaloma_AM1BCC'

In [11]:
from tqdm.notebook import tqdm

failed_interchanges = defaultdict(lambda : defaultdict(list))
for chemistry in chemistries:
    chem_path = coll_path / chemistry
    lmp_dir = lammps_path / chemistry
    lmp_dir.mkdir(exist_ok=True)

    mgr = PolymerManager(chem_path)
    for mol_name, polymer in ( progress := tqdm(mgr.polymers.items()) ):
        progress.set_postfix_str(f'{chemistry} : {mol_name}')
        try:
            chgr = MolCharger.subclass_registry[chg_method]()
            polymer.assert_charges_for(chgr, strict=True, return_cmol=False)

            sdf_path = polymer.structure_files_chgd[chg_method]
            cmol = polymer.charged_offmol_from_sdf(chg_method)
            offtop = Topology.from_molecules(cmol) # load topology from SDF file

            ic = forcefield.create_interchange(offtop, charge_from_molecules=[cmol])
            ic.to_lammps(lmp_dir / f'{mol_name}.lammps')
        except Exception as e:
            failed_interchanges[chemistry][e.__class__.__name__].append(mol_name)

  0%|          | 0/6 [00:00<?, ?it/s]

/home/timber/Documents/Python/openff-workspace/openff-toolkit/openff/toolkit/utils/base_wrapper.py:256: IncorrectNumConformersWarning: Molecule 'Molecule with name '' and SMILES '[H]c1c(c2c(c(c1[H])O(C([H])([H])C3(C(OC4=O56C7(=C5(O6C(C8(C(OC9=O15C6(=C1(O5C(C1(C(OC5=O%10%11C%12(=C%10(O%11C(C%10(C(OC%11=O%13%14C%15(=C%13(O%14C(C%13(C(OC%14=O%16%17C%18(=C%16(O%17C(C%16(C(OC%17=O%19%20C%21(=C%19(O%20C(C(C(OC%19=O%20%22C%23(=C%20(O%22C([H])([H])C([H])(C([H])([H])OC(=O)N([H])C([H])([H])C([H])([H])OC([H])([H])C([H])([H])OC([H])([H])C([H])([H])N([H])[H])O[H])C(=C(C(=C%23([H])O%16[H])[H])OC(C(OC(=O)N(C(C(OC(C(OC(C(N%19[H])([H])[H])([H])[H])([H])[H])([H])[H])([H])[H])([H])[H])[H])([H])C([H])([H])O[H])([H])[H])[H])[H])([H])[H])([H])O[H])([H])[H])C(=C(C(=C%21([H])O%13[H])[H])OC(C(OC(=O)N(C(C(OC(C(OC(C(N%17[H])([H])[H])([H])[H])([H])[H])([H])[H])([H])[H])([H])[H])[H])([H])C([H])([H])O[H])([H])[H])[H])[H])([H])[H])[H])([H])[H])C(=C(C(=C%18([H])O%10[H])[H])OC(C(OC(=O)N(C(C(OC(C(OC(C(N%14[H])([H])[H])

  0%|          | 0/63 [00:00<?, ?it/s]

In [13]:
help(Topology.from_molecules)

Help on method from_molecules in module openff.toolkit.topology.topology:

from_molecules(molecules: Union[openff.toolkit.topology.molecule.Molecule, List[openff.toolkit.topology.molecule.Molecule]]) -> 'Topology' method of abc.ABCMeta instance
    Create a new Topology object containing one copy of each of the specified molecule(s).
    
    Parameters
    ----------
    molecules : Molecule or iterable of Molecules
        One or more molecules to be added to the Topology
    
    Returns
    -------
    topology : Topology
        The Topology created from the specified molecule(s)



In [12]:
failed_interchanges

defaultdict(<function __main__.<lambda>()>,
            {'NIPU': defaultdict(list,
                         {'UnmatchedAtomsError': ['NIPU_8',
                           'NIPU_10',
                           'NIPU_9'],
                          'KeyError': ['NIPU_12']}),
             'urethane': defaultdict(list,
                         {'UnassignedValenceParameterException': ['urethane_4',
                           'urethane_11',
                           'urethane_30',
                           'urethane_27'],
                          'UnmatchedAtomsError': ['urethane_56',
                           'urethane_46',
                           'urethane_32',
                           'urethane_16',
                           'urethane_25',
                           'urethane_50',
                           'urethane_26',
                           'urethane_62',
                           'urethane_58',
                           'urethane_38',
                           'urethan

## Checking for successful residue covers of newly-generated PDB Topologies

In [ ]:
mgr = PolymerManager(coll_path / 'urethane')
pdir = mgr.polymers['urethane_6']
# pdir = mgr.polymers['NIPU_8']
offmol = pdir.offmol_matched(strict=False)
# pdir.offmol_matched(strict=True)

for atom in offmol.atoms:
    if not atom.metadata['already_matched']:
        print(atom.metadata)

In [ ]:
mgr = PolymerManager(coll_path / 'NIPU')

offmols = {}
unmatched = []
for mol_name, polymer in mgr.polymers.items():
    try:
        offmols[mol_name] = polymer.offmol
    except:
        unmatched.append(mol_name)

In [ ]:
unmatched

In [ ]:
for mol_name in unmatched:
    print(mol_name)
    polymer = mgr.polymers[mol_name]
    offmol = polymer.offmol_matched(strict=False)
    
    for atom in offmol.atoms:
        if not atom.metadata['already_matched']:
            print('\t', atom.metadata)

In [ ]:
offmols